<a href = https://wikidocs.net/64515> PyTorch로 시작하는 딥 러닝 입문 _ Wikidocs</a>

<a href = https://velog.io/@dltjrdud37/%ED%95%9C%EA%B5%AD%EC%96%B4-%ED%86%A0%ED%81%B0%ED%99%94by-konlpy-%EB%8B%A8%EC%96%B4%EC%9E%84%EB%B2%A0%EB%94%A9Word2Vec> 한국어 토큰화 </a>

# 전처리

In [3]:
import pandas as pd
import numpy as np

In [4]:
path = "/Users/sanghyun/Desktop/Coding/NLP/project/crawling/data/"

ly1 = pd.read_csv(path + "melon_ballad(301).csv")
ly2 = pd.read_csv(path + "melon_ballad(302).csv")
ly3 = pd.read_csv(path + "melon_ballad(303).csv")
ly4 = pd.read_csv(path + "melon_ballad(304).csv")
ly5 = pd.read_csv(path + "melon_ballad(305).csv")

In [5]:
# 병합
raw_list = [ly1,ly2,ly3,ly4,ly5]
df = pd.concat(raw_list)

# 번호 컬럼 drop
df.columns = ["ㅇㅇ","가수","제목","가사"]
df.drop(['ㅇㅇ'], axis=1, inplace=True)

# 중복 drop
df = df.drop_duplicates()

# 가수, 제목, 가사 없는거 drop
df.pop('가수')
df.pop('제목')
df.drop(df[df['가사']=='[가사 준비중]'].index, inplace=True)

# \n을 공백으로 변경
df['가사']= df['가사'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)

df

,가사
0,오래 두었지 내방 창가엔 너란 기억의 화분 많은 계절이 흐르고 꽃은 다 졌지만 그래...
2,어느샌가 노을 진 하늘 너머 마중 나온 달빛이 우릴 반겨주네요 좀 더 걸을까요 조금...
3,어떻게 전할까 널 향한 내 진심 말로는 부족해 나 이렇게 노래해 겁이 많고 불안한 ...
4,그대 없는 하루가 또 의미도 없이 지나가네요 Just stay 내 곁에서 머물러 줄...
7,만약 나를 떠난다면 사랑했었던 기억도 없을거야 가끔 내 생각이 난다면 시간이 잊게 ...
...,...
45,애써 나 웃어봐도 웃으려는게 더 슬퍼 보여 내 눈빛은 솔직해 안 슬프단 거짓말도 보...
46,자다 일어난 네 얼굴도 나에게는 보여도 좋아 술에 취해 털어 논 얘기들은 끝까지 꼭...
47,현아 날 버리지 말아요 현아 날 떠나지 말아요 현아 당신 떠나시면 이 슬픔 정말 어...
48,그때는 믿지 않았지 웃어버렸지 사랑하면서도 헤어져야 한다는 말에 아직도 믿진 않지만...


- 여기까지가 merge 이후 단계

# Tokenization

In [6]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/sanghyun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
lyrics = df.copy()
lyrics

,가사
0,오래 두었지 내방 창가엔 너란 기억의 화분 많은 계절이 흐르고 꽃은 다 졌지만 그래...
2,어느샌가 노을 진 하늘 너머 마중 나온 달빛이 우릴 반겨주네요 좀 더 걸을까요 조금...
3,어떻게 전할까 널 향한 내 진심 말로는 부족해 나 이렇게 노래해 겁이 많고 불안한 ...
4,그대 없는 하루가 또 의미도 없이 지나가네요 Just stay 내 곁에서 머물러 줄...
7,만약 나를 떠난다면 사랑했었던 기억도 없을거야 가끔 내 생각이 난다면 시간이 잊게 ...
...,...
45,애써 나 웃어봐도 웃으려는게 더 슬퍼 보여 내 눈빛은 솔직해 안 슬프단 거짓말도 보...
46,자다 일어난 네 얼굴도 나에게는 보여도 좋아 술에 취해 털어 논 얘기들은 끝까지 꼭...
47,현아 날 버리지 말아요 현아 날 떠나지 말아요 현아 당신 떠나시면 이 슬픔 정말 어...
48,그때는 믿지 않았지 웃어버렸지 사랑하면서도 헤어져야 한다는 말에 아직도 믿진 않지만...


In [8]:
# 영어 삭제
lyrics['가사'] = lyrics['가사'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
lyrics

/var/folders/t1/vscwc_091vq9zqnq6nv5gdp00000gn/T/ipykernel_76034/1399038347.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  lyrics['가사'] = lyrics['가사'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")


,가사
0,오래 두었지 내방 창가엔 너란 기억의 화분 많은 계절이 흐르고 꽃은 다 졌지만 그래...
2,어느샌가 노을 진 하늘 너머 마중 나온 달빛이 우릴 반겨주네요 좀 더 걸을까요 조금...
3,어떻게 전할까 널 향한 내 진심 말로는 부족해 나 이렇게 노래해 겁이 많고 불안한 ...
4,그대 없는 하루가 또 의미도 없이 지나가네요 내 곁에서 머물러 줄 수 없나요 혼...
7,만약 나를 떠난다면 사랑했었던 기억도 없을거야 가끔 내 생각이 난다면 시간이 잊게 ...
...,...
45,애써 나 웃어봐도 웃으려는게 더 슬퍼 보여 내 눈빛은 솔직해 안 슬프단 거짓말도 보...
46,자다 일어난 네 얼굴도 나에게는 보여도 좋아 술에 취해 털어 논 얘기들은 끝까지 꼭...
47,현아 날 버리지 말아요 현아 날 떠나지 말아요 현아 당신 떠나시면 이 슬픔 정말 어...
48,그때는 믿지 않았지 웃어버렸지 사랑하면서도 헤어져야 한다는 말에 아직도 믿진 않지만...


In [9]:
from konlpy.tag import Mecab, Hannanum, Kkma, Okt, Komoran

In [10]:
# 토큰화 함수 : 토큰 타입을 변경하고 싶으면 return 부분 수정
# Mecab..은 안쓸래... 

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

def tokenizer(row):
    return okt.morphs(row)

모델 시간<br><img src="./img/modelTime.png" width="600" height="700">

In [11]:
lyrics['lyrics_token'] = lyrics['가사'].apply(tokenizer)
lyrics

,가사,lyrics_token
0,오래 두었지 내방 창가엔 너란 기억의 화분 많은 계절이 흐르고 꽃은 다 졌지만 그래...,"[오래, 두었지, 내, 방, 창가, 엔, 너, 란, 기억, 의, 화분, 많은, 계절..."
2,어느샌가 노을 진 하늘 너머 마중 나온 달빛이 우릴 반겨주네요 좀 더 걸을까요 조금...,"[어느샌가, 노을, 진, 하늘, 너머, 마중, 나온, 달빛, 이, 우릴, 반겨주네요..."
3,어떻게 전할까 널 향한 내 진심 말로는 부족해 나 이렇게 노래해 겁이 많고 불안한 ...,"[어떻게, 전, 할까, 널, 향, 한, 내, 진심, 말, 로는, 부족해, 나, 이렇..."
4,그대 없는 하루가 또 의미도 없이 지나가네요 내 곁에서 머물러 줄 수 없나요 혼...,"[그대, 없는, 하루, 가, 또, 의미, 도, 없이, 지나가네요, 내, 곁, 에서,..."
7,만약 나를 떠난다면 사랑했었던 기억도 없을거야 가끔 내 생각이 난다면 시간이 잊게 ...,"[만약, 나를, 떠난다면, 사랑, 했었던, 기억, 도, 없을거야, 가끔, 내, 생각..."
...,...,...
45,애써 나 웃어봐도 웃으려는게 더 슬퍼 보여 내 눈빛은 솔직해 안 슬프단 거짓말도 보...,"[애써, 나, 웃어, 봐도, 웃으려는, 게, 더, 슬퍼, 보여, 내, 눈빛, 은, ..."
46,자다 일어난 네 얼굴도 나에게는 보여도 좋아 술에 취해 털어 논 얘기들은 끝까지 꼭...,"[자다, 일어난, 네, 얼굴, 도, 나, 에게는, 보여도, 좋아, 술, 에, 취해,..."
47,현아 날 버리지 말아요 현아 날 떠나지 말아요 현아 당신 떠나시면 이 슬픔 정말 어...,"[현아, 날, 버리지, 말아요, 현아, 날, 떠나지, 말아요, 현아, 당신, 떠나시..."
48,그때는 믿지 않았지 웃어버렸지 사랑하면서도 헤어져야 한다는 말에 아직도 믿진 않지만...,"[그때, 는, 믿지, 않았지, 웃어, 버렸지, 사랑, 하면서도, 헤어져야, 한다는,..."


In [13]:
print('최대 토큰 개수 : ',(max(len(i) for i in lyrics['lyrics_token'])))
print('평균 토큰 개수 : ',sum(map(len,lyrics['lyrics_token']))/len(lyrics['lyrics_token']))

최대 토큰 개수 :  373
평균 토큰 개수 :  167.2390243902439


# Embedding

- Embedding : text를 Vector로 변환시키는 방법 <br>
- <a href = "https://wikidocs.net/60854"> Word2Vec </a>

In [15]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=lyrics['lyrics_token'], vector_size = 100, window = 5, min_count= 0, workers = 4, sg = 1)

In [16]:
model.wv.vectors.shape

(5011, 100)

In [17]:
# '나'와 유사한 키워드 추출

model_result = model.wv.most_similar('나')
print(model_result)

[('마음', 0.9901757836341858), ('우리', 0.9900771975517273), ('곁', 0.9900560975074768), ('모습', 0.9896457195281982), ('세상', 0.9884582757949829), ('의', 0.9882237911224365), ('천사', 0.988219141960144), ('지금', 0.9881084561347961), ('넌', 0.9875211119651794), ('안', 0.9872139096260071)]
